#### El cliente ha pedido realizar un ajuste a las reglas implementadas hasta ahora, estos ajustes consisten en reemplazar algunas funciones hechas con groupBy por funciones Window, presta mucha atención y resuelve las siguientes actividades.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de las sesiones 06, 07

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.py

In [2]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame, Column, Window
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_8") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["06/movies", "06/ratings", "06/tags"]
df_dict = read_tmp_df(spark, names_list)

movies_df = df_dict["06/movies"]
ratings_df = df_dict["06/ratings"]
tags_df = df_dict["06/tags"]

movies_df.show(1, False)
ratings_df.show(1, False)
tags_df.show(1, False)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|rating|time               |
+-------+--------+------+-------------------+
|1      |1       |4.0   |2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|tag   |time               |
+-------+--------+------+-------------------+
|224183 |832     |acting|2017-06-05 08:20:27|
+-------+--------+------+-------------------+
only showing top 1 row



#### Actividad 1:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 2 de la sesión 07 - método calculate_rating_values" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### def calculate_rating_values_w(df: DataFrame) -> DataFrame:

In [3]:
# Ref ej7act2:
"""
def calculate_rating_values(df):
    crv_ratings_df = df.groupBy(f.col("movie_id"))\
    .agg(
        f.round(f.avg("rating"),2).alias("avg_rating"),
        f.round(f.stddev_pop("rating"),2).alias("stddev_rating"),
        f.count("rating").alias("count_rating"),
        f.min("time").alias("min_time_rating")
    )
    return crv_ratings_df
"""

'\ndef calculate_rating_values(df):\n    crv_ratings_df = df.groupBy(f.col("movie_id"))    .agg(\n        f.round(f.avg("rating"),2).alias("avg_rating"),\n        f.round(f.stddev_pop("rating"),2).alias("stddev_rating"),\n        f.count("rating").alias("count_rating"),\n        f.min("time").alias("min_time_rating")\n    )\n    return crv_ratings_df\n'

In [4]:
ratings_df.show(1)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row



In [5]:
## TU CODIGO VA EN ESTA CELDA:
# def calculate_rating_values_w(df: DataFrame) -> DataFrame:
# rating_df modificar codigo interno

window_act1 = Window.partitionBy(f.col("movie_id"))

def calculate_rating_values_w(df):
    crv_df = df.withColumn("avg_rating", f.round(f.avg("rating").over(window_act1), 2))\
            .withColumn("stddev_rating", f.round(f.stddev_pop("rating").over(window_act1), 2))\
            .withColumn("count_rating", f.count("rating").over(window_act1))\
            .withColumn("min_time_rating", f.min("time").over(window_act1))\
            .select("movie_id", "avg_rating", "stddev_rating", "count_rating", "min_time_rating")\
            .distinct() # Returns a new DataFrame containing the distinct rows in this DataFrame
    return crv_df

In [6]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
calculate_rating_values_w(ratings_df).show(2)
"""
Ejemplo de salida esperada:
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows
"""

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows



'\nEjemplo de salida esperada:\n+--------+----------+-------------+------------+-------------------+\n|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|\n+--------+----------+-------------+------------+-------------------+\n|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|\n|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|\n+--------+----------+-------------+------------+-------------------+\nonly showing top 2 rows\n'

In [7]:
#Prevalidaciones:
"""
TEST_val = calculate_rating_values_w(ratings_df)
print(len(TEST_val.columns))
print(TEST_val.count())
TEST_val.where(f.col("movie_id") == "296").show()
"""

'\nTEST_val = calculate_rating_values_w(ratings_df)\nprint(len(TEST_val.columns))\nprint(TEST_val.count())\nTEST_val.where(f.col("movie_id") == "296").show()\n'

In [8]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
rating_values_df = calculate_rating_values_w(ratings_df)

assert type(rating_values_df) == DataFrame
assert len(rating_values_df.columns) == 5
assert rating_values_df.count() == 83239

data = rating_values_df \
    .filter(f.col("movie_id") == "296") \
    .collect()[0]

assert data["movie_id"] == "296"
assert data["avg_rating"] == 4.19
assert data["stddev_rating"] == 0.95
assert data["count_rating"] == 108756
assert data["min_time_rating"] == datetime.datetime(1996, 2, 29, 10, 48, 44)

In [9]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(rating_values_df, "09/ratings")]

write_tmp_df(dfs)

#### Actividad 2:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 3 de la sesión 07 - métodos get_act_3_df1 y get_act_3_df2" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### Primer tabla (utilizando función concat_ws o concat): def get_act_3_df1_w(df: DataFrame) -> DataFrame:
- ##### Segunda tabla (utilizando función struct): def get_act_3_df2_w(df: DataFrame) -> DataFrame:

In [10]:
tags_df.show(5)

+-------+--------+--------------------+-------------------+
|user_id|movie_id|                 tag|               time|
+-------+--------+--------------------+-------------------+
| 224183|     832|              acting|2017-06-05 08:20:27|
| 224183|     832|              action|2017-06-05 08:20:01|
| 224183|     832|       Brawley Nolte|2017-06-05 08:19:31|
| 224183|     832|   conspiracy theory|2017-06-05 08:20:00|
| 224183|     832|Director: Ron Howard|2017-06-05 08:15:05|
+-------+--------+--------------------+-------------------+
only showing top 5 rows



In [11]:
#Declaración de windows:
window_ven1 = Window.partitionBy(f.col("movie_id"), f.col("tag")) #aquí antes tenía un f.upper(f.col("tag")) pero no transformaba
window_ven2 = Window.partitionBy(f.col("movie_id"))

In [12]:
# REFERENCIA:
"""
def get_act_3_df1(df):
    df_mtt = df.groupBy(f.col("movie_id"), f.upper(f.col("tag")).alias("tag"))\
        .agg(
            f.count(f.col("tag")).alias("count"),
            f.min("time").alias("min")
        ).select(
            f.col("movie_id"),
            f.concat_ws(" : ", f.col("tag"),f.col("count")).alias("tag_count"),
            f.col("min").alias("min_tag")
        )
    tabla1_df = df_mtt.groupBy(f.col("movie_id"))\
        .agg(
            f.sort_array(f.collect_set("tag_count")).alias("tag_count"),
            f.min("min_tag").alias("min_time_tag"),
        )
    return tabla1_df
"""

'\ndef get_act_3_df1(df):\n    df_mtt = df.groupBy(f.col("movie_id"), f.upper(f.col("tag")).alias("tag"))        .agg(\n            f.count(f.col("tag")).alias("count"),\n            f.min("time").alias("min")\n        ).select(\n            f.col("movie_id"),\n            f.concat_ws(" : ", f.col("tag"),f.col("count")).alias("tag_count"),\n            f.col("min").alias("min_tag")\n        )\n    tabla1_df = df_mtt.groupBy(f.col("movie_id"))        .agg(\n            f.sort_array(f.collect_set("tag_count")).alias("tag_count"),\n            f.min("min_tag").alias("min_time_tag"),\n        )\n    return tabla1_df\n'

In [13]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES NUEVAS SI ASI LO REQUIERES

#def get_act_3_df1_w(df: DataFrame) -> DataFrame:
def get_act_3_df1_w(df):
    df_mtt_w = df.withColumn("tag_count", f.concat_ws(" : ", f.upper(f.col("tag")), f.count("tag").over(window_ven1)))\
                .withColumn("min_tag", f.min("time").over(window_ven1))\
                .select("movie_id", "tag_count", "min_tag")\
                .distinct()
    tabla1_dff_w = df_mtt_w.withColumn("tag_count", f.sort_array(f.collect_set("tag_count").over(window_ven2)))\
                .withColumn("min_time_tag", f.min("min_tag").over(window_ven2))\
                .select("movie_id", "tag_count", "min_time_tag")\
                .distinct()
    return tabla1_dff_w # ... transformaciones a tags_df

act_3_df1 = get_act_3_df1_w(tags_df)

In [14]:
"""act_3_df1.show(2, False)
act_3_df1.printSchema()"""

'act_3_df1.show(2, False)\nact_3_df1.printSchema()'

In [15]:
# Referencia
"""
# ... transformaciones a tags_df
def get_act_3_df2(df):
    df_mtt2 = df.groupBy(f.col("movie_id"), f.upper(f.col("tag")).alias("tag"))\
        .agg(
            f.count(f.col("tag")).alias("count"),
            f.min("time").alias("min")
        ).select(
            f.col("movie_id"),
            f.struct(f.col("tag"),f.col("count")).alias("struct"),
            f.col("min").alias("min_tag")
        )
    tabla2_df = df_mtt2.groupBy(f.col("movie_id"))\
        .agg(
            f.sort_array(f.collect_set("struct")).alias("tag_count"),
            f.min("min_tag").alias("min_time_tag")
        )
    return tabla2_df
"""

'\n# ... transformaciones a tags_df\ndef get_act_3_df2(df):\n    df_mtt2 = df.groupBy(f.col("movie_id"), f.upper(f.col("tag")).alias("tag"))        .agg(\n            f.count(f.col("tag")).alias("count"),\n            f.min("time").alias("min")\n        ).select(\n            f.col("movie_id"),\n            f.struct(f.col("tag"),f.col("count")).alias("struct"),\n            f.col("min").alias("min_tag")\n        )\n    tabla2_df = df_mtt2.groupBy(f.col("movie_id"))        .agg(\n            f.sort_array(f.collect_set("struct")).alias("tag_count"),\n            f.min("min_tag").alias("min_time_tag")\n        )\n    return tabla2_df\n'

In [16]:
def get_act_3_df2_w(df):
    df_mtt2_w = df\
                .withColumn("tag", f.upper(f.col("tag")))\
                .withColumn("count", f.count("tag").over(window_ven1))\
                .withColumn("struct", f.struct(f.col("tag"), f.col("count")))\
                .withColumn("min_tag", f.min("time").over(window_ven1))\
                .select("movie_id", "struct", "min_tag")\
                .distinct()
    tabla2_dff_w = df_mtt2_w.withColumn("tag_count", f.sort_array(f.collect_set("struct").over(window_ven2)))\
                .withColumn("min_time_tag", f.min("min_tag").over(window_ven2))\
                .select("movie_id", "tag_count", "min_time_tag")\
                .distinct()
    return tabla2_dff_w # ... transformaciones a tags_df

act_3_df2 = get_act_3_df2_w(tags_df)

In [17]:
#Bueno
"""
def get_act_3_df2_w(df):
    df_mtt2_w = df.withColumn("struct", f.struct(f.upper(f.col("tag")), f.count("tag").over(window_ven1)))\
                .withColumn("min_tag", f.min("time").over(window_ven1))\
                .select("movie_id", "struct", "min_tag")\
                .distinct()
    tabla2_dff_w = df_mtt2_w.withColumn("tag_count", f.sort_array(f.collect_set("struct").over(window_ven2)))\
                .withColumn("min_time_tag", f.min("min_tag").over(window_ven2))\
                .select("movie_id", "tag_count", "min_time_tag")\
                .distinct()
    return tabla2_dff_w # ... transformaciones a tags_df

act_3_df2 = get_act_3_df2_w(tags_df)
"""

'\ndef get_act_3_df2_w(df):\n    df_mtt2_w = df.withColumn("struct", f.struct(f.upper(f.col("tag")), f.count("tag").over(window_ven1)))                .withColumn("min_tag", f.min("time").over(window_ven1))                .select("movie_id", "struct", "min_tag")                .distinct()\n    tabla2_dff_w = df_mtt2_w.withColumn("tag_count", f.sort_array(f.collect_set("struct").over(window_ven2)))                .withColumn("min_time_tag", f.min("min_tag").over(window_ven2))                .select("movie_id", "tag_count", "min_time_tag")                .distinct()\n    return tabla2_dff_w # ... transformaciones a tags_df\n\nact_3_df2 = get_act_3_df2_w(tags_df)\n'

In [18]:
"""act_3_df2.show(2, False)
act_3_df2.where(f.col("movie_id") == '100070').show(1,False)
print(len(act_3_df2.columns)) # == 3
print(act_3_df2.count()) # == 53452
act_3_df2.printSchema()"""

'act_3_df2.show(2, False)\nact_3_df2.where(f.col("movie_id") == \'100070\').show(1,False)\nprint(len(act_3_df2.columns)) # == 3\nprint(act_3_df2.count()) # == 53452\nact_3_df2.printSchema()'

In [19]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
act_3_df1.show(2, False)
act_3_df2.show(2, False)
"""
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |2018-05-26 16:40:54|
|100070  |[{COMEDIAN, 2}, {COMEDY, 1}, {GOOD HUMOUR, 1}, {STRUGGLING CAREER, 1}]|2017-05-19 17:17:36|
+--------+----------------------------------------------------------------------+-------------------+
only showing top 2 rows
"""

+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 17:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 18:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |

'\nEjemplo de salida esperada:\n+--------+------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                         |min_time_tag       |\n+--------+------------------------------------------------------------------+-------------------+\n|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|\n|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|\n+--------+------------------------------------------------------------------+-------------------+\nonly showing top 2 rows\n\n+--------+----------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                             |min_time_tag       |\n+--------+----------------------------------------------------------------------+-------------------+\n|100062  |[{FATE, 1}, {PRESS-GANGED, 1

In [20]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
expected_df1 = [
    Row(movie_id='100070',
        tag_count=['COMEDIAN : 2', 'COMEDY : 1', 'GOOD HUMOUR : 1', 'STRUGGLING CAREER : 1'],
        min_time_tag=datetime.datetime(2017, 5, 19, 17, 17, 36))]
schema_df1 = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("tag_count", t.ArrayType(t.StringType())),
    t.StructField("min_time_tag", t.TimestampType())
])
assert len(act_3_df1.columns) == 3
assert "movie_id" in act_3_df1.columns
assert "tag_count" in act_3_df1.columns
assert "min_time_tag" in act_3_df1.columns
assert schema_to_ddl(spark, act_3_df1.select("movie_id", "tag_count", "min_time_tag")).replace(" NOT NULL", "") == 'movie_id STRING,tag_count ARRAY<STRING>,min_time_tag TIMESTAMP'
assert act_3_df1.count() == 53452
assert act_3_df1.filter(f.col("movie_id") == "100070").subtract(spark.createDataFrame(expected_df1, schema_df1)).count() == 0


expected_df2 = [
    Row(movie_id='100070',
        tag_count=[Row(tag='COMEDIAN', count=2),
                   Row(tag='COMEDY', count=1),
                   Row(tag='GOOD HUMOUR', count=1),
                   Row(tag='STRUGGLING CAREER', count=1)],
        min_time_tag=datetime.datetime(2017, 5, 19, 17, 17, 36))]
schema_df2 = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("tag_count", t.ArrayType(t.StructType([
        t.StructField("tag", t.StringType()),
        t.StructField("count", t.LongType())
    ]))),
    t.StructField("min_time_tag", t.TimestampType())
])
assert len(act_3_df2.columns) == 3
assert "movie_id" in act_3_df2.columns
assert "tag_count" in act_3_df2.columns
assert "min_time_tag" in act_3_df2.columns
assert schema_to_ddl(spark, act_3_df2.select("movie_id", "tag_count", "min_time_tag")).replace(" NOT NULL", "") == 'movie_id STRING,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>>,min_time_tag TIMESTAMP'
assert act_3_df2.count() == 53452
assert act_3_df2.filter(f.col("movie_id") == "100070").subtract(spark.createDataFrame(expected_df2, schema_df2)).count() == 0

In [21]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(act_3_df1, "09/tags_p1"),
       (act_3_df2, "09/tags_p2")]

write_tmp_df(dfs)

In [22]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
last_movies_df = movies_df \
    .join(act_3_df2, ["movie_id"], "left") \
    .join(rating_values_df, ["movie_id"], "left")

In [23]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(last_movies_df, "09/movies")]

write_tmp_df(dfs)

#### Actividad 3:
##### TO DO ->    La última actividad a realizar es la siguiente:
- ##### El cliente nos ha solicitado generar una tabla donde se muestre el top de peliculas ranqueadas por genero, para realizar esto se realizan los siguientes pasos:
    - ##### Las peliculas que entran a la parte del ranqueo deben cumplir con las siguientes condiciones:
        -  count_rating > 1000
        -  avg_rating >= 4.2
        -  stddev_rating < 2
    - #####  Descomponer la columna "genres" en una columna llamada "genre"
    - ##### Agregar la columna "top" donde se asignará el valor de la función "rank" de Spark tomando las siguientes caracteristicas:
        - particionar por: "genre"
        - order por: avg_rating DESC, stddev_rating ASC, count_rating DESC
##### El esquema resultante deberá ser:
* |-- title: string
* |-- year: integer
* |-- genre: string
* |-- top: integer

In [24]:
# Tabla inicial:
last_movies_df.show(2)

+--------+--------------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|movie_id|               title|              genres|year|           tag_count|       min_time_tag|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+--------------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|       3|Grumpier Old Men ...|   [Comedy, Romance]|1995|[{ANN MARGARET, 1...|2006-01-16 00:28:45|      3.17|         1.02|       15820|1996-01-29 17:54:45|
|       1|    Toy Story (1995)|[Adventure, Anima...|1995|[{1990S, 1}, {200...|2006-01-12 19:19:35|      3.89|         0.93|       76813|1996-01-28 18:00:00|
+--------+--------------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
only showing top 2 rows



In [25]:
# Referencia:
"""
def get_all_genres(df):
    gag_df = df\
        .select(
            f.explode(f.col("genres")).alias("list_of_genres")
        )\
        .distinct()
    return gag_df
"""

'\ndef get_all_genres(df):\n    gag_df = df        .select(\n            f.explode(f.col("genres")).alias("list_of_genres")\n        )        .distinct()\n    return gag_df\n'

In [26]:
## TU CODIGO VA EN ESTA CELDA:
# Transformaciones a last_movies_df

df_act3 = last_movies_df\
    .filter(
        (f.col("count_rating") > 1000) &
        (f.col("avg_rating") >= 4.2) &
        (f.col("stddev_rating") < 2)
    )\
    .select(
        f.col("title"),
        f.col("year"),
        f.col("avg_rating"),
        f.col("stddev_rating"),
        f.col("count_rating"),
        f.explode(f.col("genres")).alias("genre")
    )

#TEST
#df_act3.show(5)

window_act3 = Window\
    .partitionBy(f.col("genre"))\
    .orderBy(
        f.col("avg_rating").desc(),
        f.col("stddev_rating").asc(),
        f.col("count_rating").desc()
    )

top_movies_df = df_act3\
    .withColumn("top", f.rank().over(window_act3))\
    .select("title", "year", "genre", "top")

#TEST:
#top_movies_df.show(5)

In [27]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

top_movies_df.show(5)
"""
Ejemplo de salida esperada (hay que mantener mismo orden de columnas):
+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows
"""

+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows



'\nEjemplo de salida esperada (hay que mantener mismo orden de columnas):\n+--------------------+----+---------+---+\n|               title|year|    genre|top|\n+--------------------+----+---------+---+\n|Band of Brothers ...|2001|   Action|  1|\n|Seven Samurai (Sh...|1954|   Action|  2|\n|   Fight Club (1999)|1999|   Action|  3|\n|Over the Garden W...|2013|Adventure|  1|\n|Seven Samurai (Sh...|1954|Adventure|  2|\n+--------------------+----+---------+---+\nonly showing top 5 rows\n'

In [28]:
assert top_movies_df.orderBy(f.col("genre").asc(), f.col("top").asc()).collect() == top_movies

In [29]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(top_movies_df, "09/top_movies")]

write_tmp_df(dfs)